In [35]:
from utils import build_graph_from_edgelist
import pandas as pd 
import numpy as np 

In [36]:
graph = build_graph_from_edgelist("graph/europe-airports.edgelist")

In [37]:
nodes = list(graph.nodes())
degrees = [graph.degree(node) for node in nodes]
degrees


[20,
 17,
 19,
 35,
 101,
 89,
 138,
 89,
 112,
 144,
 75,
 84,
 65,
 46,
 42,
 97,
 98,
 83,
 131,
 107,
 202,
 72,
 116,
 167,
 64,
 43,
 136,
 166,
 12,
 53,
 75,
 89,
 40,
 88,
 81,
 143,
 156,
 100,
 149,
 97,
 117,
 84,
 19,
 80,
 33,
 117,
 43,
 68,
 1,
 38,
 84,
 43,
 48,
 64,
 128,
 114,
 76,
 50,
 66,
 27,
 82,
 101,
 37,
 61,
 51,
 120,
 31,
 58,
 35,
 51,
 88,
 61,
 80,
 62,
 51,
 51,
 21,
 42,
 71,
 113,
 86,
 7,
 101,
 44,
 42,
 20,
 36,
 63,
 43,
 29,
 50,
 56,
 23,
 34,
 111,
 48,
 42,
 33,
 61,
 52,
 15,
 41,
 40,
 51,
 53,
 63,
 85,
 35,
 12,
 19,
 45,
 7,
 17,
 8,
 23,
 6,
 44,
 43,
 39,
 64,
 27,
 60,
 45,
 12,
 8,
 23,
 40,
 1,
 2,
 24,
 38,
 53,
 49,
 37,
 47,
 10,
 41,
 29,
 73,
 53,
 31,
 46,
 72,
 72,
 6,
 15,
 56,
 35,
 38,
 9,
 6,
 36,
 30,
 21,
 70,
 61,
 26,
 12,
 27,
 53,
 20,
 14,
 39,
 67,
 6,
 36,
 25,
 35,
 40,
 47,
 33,
 25,
 52,
 38,
 40,
 29,
 10,
 28,
 32,
 34,
 6,
 15,
 38,
 38,
 18,
 18,
 17,
 30,
 8,
 30,
 38,
 25,
 28,
 18,
 32,
 43,
 18,
 16,


In [38]:
labels = pd.read_csv("graph/labels-europe-airports.txt",delimiter=" ")
y = np.array(labels['label'].tolist())
nodes = labels["node"].tolist()
X = np.array([graph.degree(node) for node in nodes]).reshape(-1,1)

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
classifier = LogisticRegression(
        random_state=42,
        penalty="l2",
        solver="liblinear",
        multi_class="ovr",
    )
    # Perform 5-fold cross-validation
mean_accuracies = []
for i in range(5):
    scores = cross_val_score(classifier, X, y, cv=5, scoring="accuracy")
    mean_accuracy = np.mean(scores)
    mean_accuracies.append(mean_accuracy)

In [40]:
overall_mean_accuracy = np.mean(mean_accuracies)
overall_mean_accuracy

0.49591772151898744

In [7]:
# 导入嵌入
from gensim.models import Word2Vec

In [43]:
model = Word2Vec.load("model_save/europe_word2vec.model")
nodes = sorted(labels["node"].tolist())
nodes_str = [str(node) for node in nodes]
embeddings = [model.wv[node] for node in nodes_str]
embeddings = np.stack(embeddings, axis=0)

In [44]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
X = embeddings
y = np.array(labels["label"].tolist())
# Split the data into training and testing sets
# Standardize the embeddings
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train a OneVsRest logistic regression classifier with L2 regularization
# classifier = OneVsRestClassifier(
#     LogisticRegression(
#         random_state=42, penalty="l2", tol=0.001, C=1, solver="liblinear"
#     )
# )
# classifier = OneVsRestClassifier(GradientBoostingClassifier())
classifier = LogisticRegression(
    random_state=42,
    penalty="l2",
    tol=0.0001,
    C=3,
    solver="liblinear",
    multi_class="ovr",
)
# Perform 5-fold cross-validation
# mean_accuracies = []
# for i in range(5):
#     scores = cross_val_score(classifier, X, y, cv=5, scoring="accuracy")
#     mean_accuracy = np.mean(scores)
#     mean_accuracies.append(mean_accuracy)

# # Calculate and print the overall mean accuracy
# overall_mean_accuracy = np.mean(mean_accuracies)
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'tol': [0.0001, 0.001, 0.01,0.00001],
    'solver': ['liblinear']
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score

({'C': 0.1, 'solver': 'liblinear', 'tol': 0.01}, 0.536139240506329)